# Homework 9:

Repeat the procedure we did in class to map the cities of the US using a map and the results of a PCoA, but using a different country of your choosing.

Tips:

These are the packages that you will need:

```
import geopy.distance
import pandas as pd
import plotly.express as px
import skbio
```

You will also need the `world_cities.csv` file to get the city names and coordinates for your country.

1. Choose your country.
2. Get only the cities of that country from the `world_cities.csv` file.
3. Sort the cities by population and select the 20 to 50 most populated cities.
4. Create a data frame with only these cities.
5. Calculate the distance between the cities using their latitude/longitude data from your data frame (geopy).
6. Create a data frame with a table of all the distances between each paur of cities. This needs to be in a distance matrix format, just like we imported from the `cities.csv` file for the US.
7. Perform the PCoA using skbio.
8. Plot the results of the PCoA using plotly express.
9. Plot the map of the country with the cities using the latitude/longitude from your table using plotly express.
10. That's it!

In [55]:
from geopy.distance import geodesic
import pandas as pd
import plotly.express as px
from skbio.stats.ordination import pcoa

df = pd.read_csv("world_cities.csv",sep="\t")
df = df.loc[df['country.etc'] == "China"]
df = df.sort_values(by=['pop'], ascending=False)
df = df.head(30)

def ellipsoidal_distance(p1, p2) -> float:
    return geodesic(p1, p2).meters

# dist = pd.DataFrame({
#     'lat': df['lat'].tolist(), 
#     'long': df['long'].tolist()
#     })

distmatrix = pd.DataFrame(index=df.name, columns=df.name)
coords = df.loc[:, ["lat", "long"]]

for lat1, long1 in coords.iterrows():  # for each origin
    for lat2, long2 in coords.iterrows():  # for each destination
        distmatrix.at[name, name] = ellipsoidal_distance(, dest_loc)

distmatrix

# geopy.distance()
# fig = px.scatter_geo(df, lon = 'long', lat = 'lat',text="name", size="pop")
# fig.update_layout(geo_scope="China")
# my_pcoa = pcoa.pcoa(df)
# fig = px.scatter(my_pcoa['loadings'], x = "PC1", y = "PC2", text=list(df.columns))

# fig.update_xaxes(autorange="reversed")
# fig.update_traces(textposition="bottom right")
# fig

name,Shanghai,Beijing,Wuhan,Chongqing,Xian,Chengdu,Tianjin,Shenyang,Harbin,Nanjing,...,25335,13837,30367,42029,37328,39657,20223,11970,14133,36723
name,,,,,,,,,,,,,,,,,,,,,
Shanghai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Beijing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wuhan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chongqing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Xian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chengdu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tianjin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Shenyang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Harbin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
